# Bug Detection

In [1]:
import os
import re
import io
import sys
import wget
import pickle
import tarfile
import itertools
import functools
import subprocess

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from IPython.display import display, HTML
from tqdm import tqdm
from pprint import pprint
from difflib import Differ, SequenceMatcher

tqdm.pandas()

# Initialize the dataset paths
input_path = '../input/'
root_path = input_path + 'Project_CodeNet/'

os.makedirs(input_path, exist_ok=True)

data_path = root_path + 'data/'
metadata_path = root_path + 'metadata/'
derived_path = root_path + 'derived/'
descriptions_path = root_path + 'problem_descriptions/'

# Create helper functions to find the path easier in the dataset
def id2desc(problem_id): return descriptions_path + problem_id + '.html'
def id2inout(problem_id, name='input'): 
    return derived_path + 'input_output/data/' + problem_id + '/' + name + '.txt'
def id2submission(problem_id, language, submission_id, filename_ext): 
    return data_path + problem_id + '/' + language + '/' + submission_id + '.' + filename_ext

# Download CodeNet

The next code cell will download the CodeNet dataset from it's original repository (the archive has around 80GB). If you already have the dataset change the input_path variable to point to the root of the dataset, otherwise the notebook will download it in the ../input/ directory.

In [2]:
data_url = "https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0"
tar_name = "Project_CodeNet.tar.gz"
tar_path = input_path + tar_name

def download_data():
    if os.path.exists(root_path):
        print("dataset root dir found")
        return

    if not os.path.exists(tar_path):
        wget.download(f"{data_url}/{tar_name}", out=tar_path)
        
    with tarfile.open(tar_path) as tf:
        tf.extractall(path=data_path)

download_data()

dataset root dir found


# Data Exploration

The base dataset I will be using in this notebook is [CodeNet](https://github.com/IBM/Project_CodeNet) which is a large collection of source files and problem descriptions with metadata. The solutions are written in multiple programming languages (55+ according to the paper) and each problem has multiple submissions. Most of the submissions are written in the six most common languages (C++, Python, Java, C, Ruby, C#). As expected most of the solutions are in C++. One interesting aspect of the dataset is that it includes failed submissions, with various status codes such as Compilation Errors, Runtime Errors, Time Limit Exceeded, Memory Limit Exceeded, etc. This will prove useful since we are looking into bug detection in source code files. The dataset also contains some sample input and output for some of the problems, so we can extract even lower level errors for the submissions, for example the Runtime Error of a specific submission may refer to a IndexError, or even more specific, IndexError: Index {number} is out of bounds. Another important aspect of the bug detection task is pointing to the location in code where the error occurs, so the fact that we have sample inputs and outputs is again a great feature. Another idea that can be used to find erroneous instructions is to compute the difference between two consecutive submissions for a single user. This can also be done since the metadata files of each problem keep track of user ids and submission ids. In the end a good state for the dataset is to have as many source files as possible labeled with the location in code where a specific error occurs.

Let's load the problem list table and look at the data. We are interested in seeing how many problems we have and from what coding competition each problem is taken from.

In [3]:
# Load the problem list table as a csv
problem_list_df = pd.read_csv(metadata_path + 'problem_list.csv')
problem_list_df.set_index('id', inplace=True)

print(f"We have {len(problem_list_df)} problems")
print('The distribution of the datasets is')
print(problem_list_df['dataset'].value_counts(normalize=True))
display(problem_list_df.head())
display(problem_list_df.isna().sum())

We have 4053 problems
The distribution of the datasets is
AIZU       0.625216
AtCoder    0.374784
Name: dataset, dtype: float64


,name,dataset,time_limit,memory_limit,rating,tags,complexity
id,,,,,,,
p00000,QQ,AIZU,1000.0,131072.0,NaN,NaN,NaN
p00001,List of Top 3 Hills,AIZU,1000.0,131072.0,NaN,NaN,NaN
p00002,Digit Number,AIZU,1000.0,131072.0,NaN,NaN,NaN
p00003,Is it a Right Triangle?,AIZU,1000.0,131072.0,NaN,NaN,NaN
p00004,Simultaneous Equation,AIZU,1000.0,131072.0,NaN,NaN,NaN


name              56
dataset            0
time_limit        56
memory_limit      56
rating          4053
tags            4053
complexity      4053
dtype: int64

In [4]:
problem_list_df['time_limit'].fillna(1.0, inplace=True)

We can see that we have around 4000 problems with 2 thirds coming from the AIZU dataset and the rest from the AtCoder dataset. Another interesting aspect is that the rating, tags and complexity is missing for all the problems. The tags could have beed useful to classify the problem into corresponding categories, such as graph problem or dynamic programing problem, etc. There are also some data missing for the name, time limit and memory limit for a few problems, but it shouldn't be a problem. Let's look for example at the first problem and see how many submissions it has, what programming languages were used, what was the distribution of the status for the submissions and the users that attempted this problem. Let's also see what was the average number of submissions per user.

In [5]:
# Load the problem metadata
problem_ids = problem_list_df.index.unique()
problem_id = 'p00000'

problem_df = pd.read_csv(metadata_path + f'{problem_id}.csv')
problem_df.set_index('submission_id', inplace=True)

print(f"We have {len(problem_df)} submissions for problem {problem_id}")
print('The distribution of the programming languages is')
print(problem_df['language'].value_counts())
print('The distribution of the status is')
print(problem_df['status'].value_counts())
print('The number of users that attempted this problem', len(problem_df['user_id'].unique()))
print('The average number of submissions of a user', problem_df.groupby('user_id').size().mean())
print('The min number of submissions of a user', problem_df.groupby('user_id').size().min())
print('The median number of submissions of a user', problem_df.groupby('user_id').size().median())
print('The max number of submissions of a user', problem_df.groupby('user_id').size().max())
display(problem_df.head())

We have 16099 submissions for problem p00000
The distribution of the programming languages is
C++           5231
C             4849
Java          2291
Python        1626
Ruby           701
C#             416
PHP            316
JavaScript     288
Scala          105
Haskell        103
D               89
Go              31
Rust            24
OCaml           22
Kotlin           7
Name: language, dtype: int64
The distribution of the status is
Accepted                  8597
Wrong Answer              3187
Compile Error             2638
Runtime Error              952
WA: Presentation Error     545
Time Limit Exceeded        164
Memory Limit Exceeded       13
Output Limit Exceeded        3
Name: status, dtype: int64
The number of users that attempted this problem 5944
The average number of submissions of a user 2.708445491251682
The min number of submissions of a user 1
The median number of submissions of a user 1.0
The max number of submissions of a user 1362


,problem_id,user_id,date,language,original_language,filename_ext,status,cpu_time,memory,code_size,accuracy
submission_id,,,,,,,,,,,
s694813024,p00000,u706566315,1540908251,Rust,Rust,rs,Wrong Answer,0,5012,120,0/1
s554950692,p00000,u706566315,1540908314,Rust,Rust,rs,Accepted,0,5004,124,1/1
s555203498,p00000,u759934006,1513417513,Rust,Rust,rs,Accepted,0,5020,136,1/1
s309783173,p00000,u233505136,1516826051,Rust,Rust,rs,Accepted,0,5004,123,1/1
s184977351,p00000,u191088660,1517740150,Rust,Rust,rs,Accepted,0,5004,122,1/1


We can see that for this problem the top six used programming languages matches the top six of the entire dataset (C++, C, Java, Python, Ruby, C#) and also that the distribution of the status is again consistent, with around 50% accepted submissions and the rest being some form of error, according to the dataset paper. It looks like the number of users that tried this problem is very large, ~6000, the average number of submission being equal to 2.7, but in this case we can see that the average is not that useful, it being very sensitive to outliers, as there is an user that sent over 1000 submissions, so I think that the median is a better metric for trying to see how hard a problem was. In the case of the first problem the median number of submissions for a single user is 1, so we can assume that this problem was easy to solve. Now I think that C is a simple enough language so we can take a subsample of the first problem containing only the C solutions and continue from there.

In [6]:
# Select a subset that uses a single programming language of the submissions
languages = problem_df['language'].unique()
language = 'C'

problem_df = problem_df[problem_df['language'] == language]

print(f"We have {len(problem_df)} submissions for problem {problem_id}")
print('The distribution of the programming languages is')
print(problem_df['language'].value_counts())
print('The distribution of the status is')
print(problem_df['status'].value_counts())
print('The number of users that attempted this problem', len(problem_df['user_id'].unique()))
print('The average number of submissions of a user', problem_df.groupby('user_id').size().mean())
print('The min number of submissions of a user', problem_df.groupby('user_id').size().min())
print('The median number of submissions of a user', problem_df.groupby('user_id').size().median())
print('The max number of submissions of a user', problem_df.groupby('user_id').size().max())
display(problem_df.head())

We have 4849 submissions for problem p00000
The distribution of the programming languages is
C    4849
Name: language, dtype: int64
The distribution of the status is
Accepted                  2089
Wrong Answer              1126
Compile Error              993
Runtime Error              389
WA: Presentation Error     189
Time Limit Exceeded         61
Memory Limit Exceeded        2
Name: status, dtype: int64
The number of users that attempted this problem 1704
The average number of submissions of a user 2.845657276995305
The min number of submissions of a user 1
The median number of submissions of a user 1.0
The max number of submissions of a user 865


,problem_id,user_id,date,language,original_language,filename_ext,status,cpu_time,memory,code_size,accuracy
submission_id,,,,,,,,,,,
s113530555,p00000,u564373483,1530931912,C,C,c,Accepted,0,2036,148,1/1
s650564374,p00000,u011621222,1530986413,C,C,c,Wrong Answer,0,2044,177,0/1
s054320558,p00000,u011621222,1530987028,C,C,c,Wrong Answer,0,2044,191,0/1
s104411660,p00000,u011621222,1531006441,C,C,c,Wrong Answer,0,2032,197,0/1
s966149516,p00000,u011621222,1531030321,C,C,c,Wrong Answer,0,1984,190,0/1


Again it looks like the distribution of the statistics is consistent with the entire dataset even for this subset. Now it would be interesting to see some examples of solutions found for this problem. Let's start by creating a function that reads a C submission from the dataset source files.

In [7]:
# We can hardcode for now that the language we use is C,
# the extension is '.c' and that the problem that we are
# interested in is the first one 'p00000'.

def read_submission_file(submission):
    with open(id2submission('p00000', 'C', submission, 'c')) as f:
        text = f.readlines()

    return text

sys.stdout.writelines(read_submission_file(problem_df[problem_df['status'] == 'Accepted'].index[0]))

#include<stdio.h>

int main(){
  int i,j;

  for(i=1;i<10;i++){
    for(j=1;j<10;j++){
      printf("%dx%d=%d\n",i,j,i*j);
    }
  }

  return 0;
}


Now it looks like this submission was successful so let's try and find some users that had some difficulties and sent more than one submission.

In [8]:
counter = 0

user_ids = problem_df['user_id'].unique()
for user_id in user_ids:
    submission_df = problem_df[problem_df['user_id'] == user_id]
    
    if len(submission_df) < 2:
        continue
        
    counter += 1
    
print(f'Out of {len(user_ids)}, {counter} users have more than 1 submission')

Out of 1704, 747 users have more than 1 submission


One interesting aspect here is how many users sent more than one submission and also have submissions of the form (failed, accepted) and have only one instruction changed so that we can understand what change made their code work. To do this we need to implement a function that calls the diff utility in the linux shell. Also to check if a diff output has only one instruction changed we need to implement a grouping function.

In [9]:
def exec_diff(original_path, changed_path):
    process = subprocess.Popen(['diff', original_path, changed_path], 
                               stdout=subprocess.PIPE,
                               universal_newlines=True)
    diff, error = process.communicate()

    return diff

def group_diff_chunks(diff):
    """
    This function creates a list by extracting the start line, end line for
    the files in the diff that are of the form n[,n]cn[,n] (the output of the
    diff command in shell)
    """
    def normalize_chunk(chunk):
        s1, e1, s2, e2 = chunk

        s1 = int(s1)
        e1 = (s1 if not e1 else int(e1)) + 1
        s2 = int(s2)
        e2 = (s2 if not e2 else int(e2)) + 1

        return s1, e1, s2, e2

    p = re.compile(r'^([0-9]+),?([0-9]*)[a-zA-Z]([0-9]+),?([0-9]*)', re.MULTILINE)
    chunks = p.findall(diff)

    return [normalize_chunk(chunk) for chunk in chunks]

def single_change(chunk):
    s1, e1, s2, e2 = chunk

    return e1 - s1 == e2 - s2 == 1

In [10]:
counter = 0
total_counter = 0

user_ids = problem_df['user_id'].unique()
for user_id in user_ids:
    submission_df = problem_df[problem_df['user_id'] == user_id]

    if len(submission_df) < 2:
        continue

    submission_ids = submission_df.index.unique()
    original_id = submission_ids[0]
    for original_id, changed_id in zip(submission_ids, submission_ids[1:]):
        original_path = id2submission('p00000', 'C', original_id, 'c')
        changed_path = id2submission('p00000', 'C', changed_id, 'c')
        
        diff = exec_diff(original_path, changed_path)

        chunks = group_diff_chunks(diff)

        if len(chunks) == 1 and single_change(chunks[0]):
            original_status = submission_df.loc[original_id, 'status']
            changed_status = submission_df.loc[changed_id, 'status']

            if changed_status == 'Accepted' and original_status != 'Accepted':
                counter += 1
        
        total_counter += 1
                
print(f'Out of {total_counter}, {counter} submissions have a single instruction changed')

Out of 3145, 394 submissions have a single instruction changed


So it looks like we have some users that changed a single instruction and then managed to make their solution work. One problem with this algorithm is that it is quite slow since it needs to execute diff from python, so as a solution we can use a function already implemented from the difflib library with some wrapper methods from github: https://github.com/glanois/code I changed something from this implementation though, which was causing the function to print the delimiters with new lines intercalated https://github.com/alexjercan/code.

# Generate Source Code Pairs

### Task 1.1: Get the source code pairs
- for each problem:
- group solutions by user and select the solutions that are consecutive and of the form (Error, Accepted) with only one instruction (line) changed; heuristic to search a smaller space, since a user might submit a correct solution after a wrong one
- get the diff lines and error type
- build a df from this list and save it
- sanity check: check that the error types of this df are the same as the types in the problem metadata: Success
- done

Up until this point we played around with single files and checked the looked at how we can load the source code from the dataset. In this section we will see how we can get the diff between source code files and more specifically the instruction (given by the line in the file) that caused a problem.

In [11]:
class POSIXDiffer(Differ):
    """
    This class produces differences in the POSIX default format
    (see http://www.unix.com/man-page/POSIX/1posix/diff/),
    which is the same as the Gnu diff "normal format"
    (see http://www.gnu.org/software/diffutils/manual/diffutils.html#Normal).
    """

    def compare(self, a, b):
        cruncher = SequenceMatcher(self.linejunk, a, b)
        for tag, alo, ahi, blo, bhi in cruncher.get_opcodes():
            if alo == ahi:
                f1 = '%d' % alo
            elif alo+1 == ahi:
                f1 = '%d' % (alo+1)
            else:
                f1 = '%d,%d' % (alo+1, ahi)

            if blo == bhi:
                f2 = '%d' % blo
            elif blo+1 == bhi:
                f2 = '%d' % (blo+1)
            else:
                f2 = '%d,%d' % (blo+1, bhi)

            if tag == 'replace':
                g = itertools.chain([ '%sc%s\n' % (f1, f2) ], self._my_plain_replace(a, alo, ahi, b, blo, bhi))
            elif tag == 'delete':
                g = itertools.chain([ '%sd%s\n' % (f1, f2) ], self._dump('<', a, alo, ahi))
            elif tag == 'insert':
                g = itertools.chain([ '%sa%s\n' % (f1, f2) ], self._dump('>', b, blo, bhi))
            elif tag == 'equal':
                g = []
            else:
                raise(ValueError, 'unknown tag %r' % (tag,))

            for line in g:
                yield line

    def _my_plain_replace(self, a, alo, ahi, b, blo, bhi):
        assert alo < ahi and blo < bhi
        first  = self._dump('<', a, alo, ahi)
        second = self._dump('>', b, blo, bhi)

        for g in first, ['---\n'], second:
            for line in g:
                yield line


def pdiff(a, b):
    """
    Compare `a` and `b` (lists of strings); return a POSIX/Gnu "normal format" diff.
    """
    return POSIXDiffer().compare(a, b)


In [12]:
def read_submission_file(problem_id, language, submission_id, extension):
    """
    Read the source code as a list of lines for a given problem and submission id
    the language and extension are also required to complete the path to the file
    """
    with open(id2submission(problem_id, language, submission_id, extension)) as f:
        text = f.readlines()

    return text

In [13]:
counter = 0
total_counter = 0

user_ids = problem_df['user_id'].unique()
for user_id in user_ids:
    submission_df = problem_df[problem_df['user_id'] == user_id]

    if len(submission_df) < 2:
        continue

    submission_ids = submission_df.index.unique()
    original_id = submission_ids[0]
    for original_id, changed_id in zip(submission_ids, submission_ids[1:]):
        original_text = read_submission_file('p00000', 'C', original_id, 'c')
        changed_text = read_submission_file('p00000', 'C', changed_id, 'c')
        
        diff = pdiff(original_text, changed_text)
        diff = ''.join(diff)

        chunks = group_diff_chunks(diff)

        if len(chunks) == 1 and single_change(chunks[0]):
            original_status = submission_df.loc[original_id, 'status']
            changed_status = submission_df.loc[changed_id, 'status']

            if changed_status == 'Accepted' and original_status != 'Accepted':
                counter += 1
                
        total_counter += 1
                
print(f'Out of {total_counter}, {counter} submissions have a single instruction changed')

Out of 3145, 394 submissions have a single instruction changed


Now this runs much faster and we get the same result as with using the diff tool. Great so now we can easily find two consecutive submissions and see the instruction that was changed to make a status changed from failed to success.

In [14]:
def preprocess_problem_for_language(problem_df, problem_id, language='C', extension='c'):
    """
    Given the problem metadata dataframe, the problem id (which is just the id from the
    dataframe), tha programming language and extension for the filepath
    iterate over all the users and group their solutions in smaller tables
    Then iterate over the submissions with a window of size 2 and
    if there are two consecutive attempts that can form a pair
    (error, accepted) and the difference between the source files
    is just a single instruction, add this pair to the result
    """
    result = []

    user_ids = problem_df['user_id'].unique()
    for user_id in user_ids:
        submission_df = problem_df[problem_df['user_id'] == user_id]

        if len(submission_df) < 2:
            continue

        submission_ids = submission_df.index.unique()
        original_id = submission_ids[0]
        for original_id, changed_id in zip(submission_ids, submission_ids[1:]):
            original_text = read_submission_file(problem_id, language, original_id, extension)
            changed_text = read_submission_file(problem_id, language, changed_id, extension)

            diff = pdiff(original_text, changed_text)
            diff = ''.join(diff)

            chunks = group_diff_chunks(diff)

            if len(chunks) == 1 and single_change(chunks[0]):
                original_status = submission_df.loc[original_id, 'status']
                changed_status = submission_df.loc[changed_id, 'status']
                original_language = submission_df.loc[changed_id, 'original_language']

                if changed_status == 'Accepted' and original_status != 'Accepted':
                    result.append((
                        original_id,
                        changed_id,
                        chunks[0][0],
                        chunks[0][2],
                        original_status,
                        original_language
                    ))

    return result

In [15]:
result = preprocess_problem_for_language(problem_df, problem_id='p00000', language='C', extension='c')
(original_id, changed_id, original_start, changed_start, original_status, original_language) = result[0]
result[0]

('s381898998', 's865898267', 9, 9, 'Wrong Answer', 'C')

In the result we can see the two submission ids and the ranges of the change in the file. The line numbers start from 1 (the usual convention for text files) and should act as a range (start, end not included). Now that we have an example of a solution that had a single instruction changed let's visualize the diff.

In [16]:
original_text = read_submission_file('p00000', 'C', original_id, 'c')
changed_text = read_submission_file('p00000', 'C', changed_id, 'c')

diff = pdiff(original_text, changed_text)
sys.stdout.writelines(diff)

9c9
<       printf("%dX%d=%d\n", n, i, n*i);
---
>       printf("%dx%d=%d\n", n, i, n*i);


In this example we can see that the diff in both files starts and ends on the 9th line of the file, this is reflected in the ranges from the result (9, 10), (9, 10). This example looks like a spelling error, where the user though that the product symbol for this problem output was a capital X instead of a lower case x. With some more attention this error could have been caught if the description had some output examples or constraints indicators. So let's take a look a the description file for this example.

In [17]:
with open(id2desc('p00000')) as f:
    text = f.readlines()
display(HTML(''.join(text)))

From this example we can see that the output sample gave some hints towards why the first solution did not work, it should have used lower case x. Now let's look for some other interesting error cases. First let's convert the result array to a pandas DataFrame for faster searching.

In [18]:
columns = ['original_id', 'changed_id', 'original_start', 'changed_start', 'original_status', 'original_language']

df = pd.DataFrame(result, columns=columns)
df['problem_id'] = 'p00000'
df['language'] = 'C'
df['filename_ext'] = 'c'

df.head()

,original_id,changed_id,original_start,changed_start,original_status,original_language,problem_id,language,filename_ext
0,s381898998,s865898267,9,9,Wrong Answer,C,p00000,C,c
1,s749240628,s634652749,11,11,Wrong Answer,C,p00000,C,c
2,s607203127,s502818484,9,9,Wrong Answer,C,p00000,C,c
3,s719001213,s042257533,9,9,Wrong Answer,C,p00000,C,c
4,s066960572,s577188800,10,10,Wrong Answer,C,p00000,C,c


In [19]:
cdf = df[df['original_status'] == 'Compile Error']

print(f'There are {len(cdf)} submissions that first had compilation errors')

There are 50 submissions that first had compilation errors


In [20]:
(original_id, changed_id, original_start, 
 changed_start, original_status, original_language, problem_id, language, filename_ext) = cdf.iloc[0]

original_text = read_submission_file(problem_id, language, original_id, filename_ext)
changed_text = read_submission_file(problem_id, language, changed_id, filename_ext)

diff = pdiff(original_text, changed_text)
sys.stdout.writelines(diff)

6a7
> }


This compile error looks like it was caused by a missing bracket on line 6 in the original file. Let's also look at that file source code.

In [21]:
sys.stdout.writelines(original_text)

#include <stdio.h>
int main(void){
    for(int i=1;i<=9;i++){
        for(int j=1;j<=9;j++){
        printf("%dx%d=%d\n",i,j,i*j);
    }
    return 0;
}



At first the submission looks fine but the problem is the formatting which creates the illusion that everything should work. Let's try to use the gcc compiler and see how this error would look like.

In [22]:
def exec_gcc(file_path, out='a.out'):
    process = subprocess.Popen(['gcc', file_path], 
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               universal_newlines=True)
    output, error = process.communicate()

    return output, error

file_path = id2submission(problem_id, language, original_id, filename_ext)
sys.stdout.writelines(exec_gcc(file_path)[1])

../input/Project_CodeNet/data/p00000/C/s937280655.c: In function ‘main’:
../input/Project_CodeNet/data/p00000/C/s937280655.c:8:1: error: expected declaration or statement at end of input
    8 | }
      | ^


This doesn't look like the best error message given the formatting of the original file, but at least it hints towards a missing closing bracket.

In [23]:
cdf = df[df['original_status'] == 'Runtime Error']

print(f'There are {len(cdf)} submissions that first had runtime errors')

There are 73 submissions that first had runtime errors


In [24]:
(original_id, changed_id, original_start, 
 changed_start, original_status, original_language, problem_id, language, filename_ext) = cdf.iloc[0]

original_text = read_submission_file(problem_id, language, original_id, filename_ext)
changed_text = read_submission_file(problem_id, language, changed_id, filename_ext)

diff = pdiff(original_text, changed_text)
sys.stdout.writelines(diff)

8a9
>     return 0;


This compile error looks like it was caused by a missing return statement on line 8 in the original file. Let's also look at that file source code.

In [25]:
sys.stdout.writelines(original_text)

#include<stdio.h>
int main(){
    int i,j;
    for(i=1;i<=9;i++){
        for(j=1;j<=9;j++){
            printf("%dx%d=%d\n",i,j,i*j);
        }
    }
}

In [30]:
def exec_gcc(file_path, out='a.out'):
    process = subprocess.Popen(['gcc', '-std=c90', file_path], 
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               universal_newlines=True)
    output, error = process.communicate()

    return output, error

file_path = id2submission(problem_id, language, original_id, filename_ext)
sys.stdout.writelines(exec_gcc(file_path)[1])

Because this is a runtime error then the compilation should be successful and indeed it is. So now let's try to run the problem. Since this problem didn't have a need for an input from stdin we can just run it as is.

In [31]:
exec_gcc(file_path)
process = subprocess.Popen(['./a.out'], 
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           universal_newlines=True)
output, error = process.communicate()

!rm './a.out'
output, error, process.returncode

('1x1=1\n1x2=2\n1x3=3\n1x4=4\n1x5=5\n1x6=6\n1x7=7\n1x8=8\n1x9=9\n2x1=2\n2x2=4\n2x3=6\n2x4=8\n2x5=10\n2x6=12\n2x7=14\n2x8=16\n2x9=18\n3x1=3\n3x2=6\n3x3=9\n3x4=12\n3x5=15\n3x6=18\n3x7=21\n3x8=24\n3x9=27\n4x1=4\n4x2=8\n4x3=12\n4x4=16\n4x5=20\n4x6=24\n4x7=28\n4x8=32\n4x9=36\n5x1=5\n5x2=10\n5x3=15\n5x4=20\n5x5=25\n5x6=30\n5x7=35\n5x8=40\n5x9=45\n6x1=6\n6x2=12\n6x3=18\n6x4=24\n6x5=30\n6x6=36\n6x7=42\n6x8=48\n6x9=54\n7x1=7\n7x2=14\n7x3=21\n7x4=28\n7x5=35\n7x6=42\n7x7=49\n7x8=56\n7x9=63\n8x1=8\n8x2=16\n8x3=24\n8x4=32\n8x5=40\n8x6=48\n8x7=56\n8x8=64\n8x9=72\n9x1=9\n9x2=18\n9x3=27\n9x4=36\n9x5=45\n9x6=54\n9x7=63\n9x8=72\n9x9=81\n',
 '',
 7)

In this example the program outputs just fine the correct answer but the checker also looked at the return code, and since the original file didn't have a return 0 statement then the return code of the program was undefined and didn't allow for consistent behaviour which could lead to errors. Also from this example it looks like the gcc is used with the standard 90 C since the default gcc command outputs 0 as the return command otherwise.

Now that we have all the missing pieces we can create a table for all the problems and languages in this dataset that are in the same format, one instruction changed that fixed the submission. 

In [36]:
generated_pairs_path = input_path + 'data.csv'

In [29]:
output = generated_pairs_path

problem_list_df = pd.read_csv(metadata_path + 'problem_list.csv')
problem_list_df.set_index('id', inplace=True)
problem_ids = problem_list_df.index.unique()

columns = ['original_id', 'changed_id', 'original_line', 'changed_line', 'original_status', 'original_language']
dfs = []

loop = tqdm(problem_ids)
for problem_id in loop:
    loop.set_description("Processing %s" % problem_id)

    problem_df = pd.read_csv(metadata_path + f'{problem_id}.csv')
    if problem_df.empty:
        continue

    problem_df.set_index('submission_id', inplace=True)

    grouped_languages = problem_df.groupby('language')
    for language, problem_df in grouped_languages:
        if problem_df.empty:
            continue

        extension = problem_df.iloc[0]['filename_ext']
        xs = preprocess_problem_for_language(problem_df, problem_id, language, extension)
        df = pd.DataFrame(xs, columns=columns)
        df['problem_id'] = problem_id
        df['language'] = language
        df['filename_ext'] = extension
        dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df.to_csv(output, index=False)

Processing p04052: 100%|███████████████████████████████████████████████████████| 4053/4053 [2:26:48<00:00,  2.17s/it]


In [34]:
display(df)
display(df.info())
display(df.language.value_counts())
display(df.original_status.value_counts())

,original_id,changed_id,original_line,changed_line,original_status,original_language,problem_id,language,filename_ext
0,s381898998,s865898267,9,9,Wrong Answer,C,p00000,C,c
1,s749240628,s634652749,11,11,Wrong Answer,C,p00000,C,c
2,s607203127,s502818484,9,9,Wrong Answer,C,p00000,C,c
3,s719001213,s042257533,9,9,Wrong Answer,C,p00000,C,c
4,s066960572,s577188800,10,10,Wrong Answer,C,p00000,C,c
...,...,...,...,...,...,...,...,...,...
102551,s753736051,s069032276,59,59,Time Limit Exceeded,C++14 (GCC 5.4.1),p04049,C++,cpp
102552,s937763329,s353716814,46,46,Wrong Answer,C++ (GCC 9.2.1),p04050,C++,cpp
102553,s181482285,s112746800,29,29,Wrong Answer,C++14 (GCC 5.4.1),p04050,C++,cpp
102554,s515906114,s712940997,42,42,Wrong Answer,C++ (GCC 5.4.1),p04051,C++,cpp


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102556 entries, 0 to 102555
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   original_id        102556 non-null  object
 1   changed_id         102556 non-null  object
 2   original_line      102556 non-null  int64 
 3   changed_line       102556 non-null  int64 
 4   original_status    102556 non-null  object
 5   original_language  102556 non-null  object
 6   problem_id         102556 non-null  object
 7   language           102556 non-null  object
 8   filename_ext       102556 non-null  object
dtypes: int64(2), object(7)
memory usage: 7.0+ MB


None

C++             49996
C               19609
Python          11678
Java             9746
Ruby             3652
C#               2065
Perl              815
Haskell           734
JavaScript        698
Awk               523
PHP               511
Scala             409
dc                398
D                 360
Rust              211
Go                206
Octave            148
OCaml             144
Bash              143
Kotlin             77
Vim                70
Sed                67
Nim                48
Swift              40
Julia              39
Bf                 38
Crystal            22
Pascal             20
Lisp               18
Fortran            14
Scheme             11
Lua                 9
Cython              9
Dash                6
TypeScript          6
bc                  4
F#                  3
Racket              2
Visual Basic        2
Clojure             2
COBOL               1
MoonScript          1
Unlambda            1
Name: language, dtype: int64

Wrong Answer              58169
Compile Error             15134
Runtime Error             13132
WA: Presentation Error    11505
Time Limit Exceeded        4099
Memory Limit Exceeded       487
Output Limit Exceeded        29
Judge Not Available           1
Name: original_status, dtype: int64

In [35]:
# sanity check (check that the status code is correctly copied for each submission)
def check(pid):
    problem_df = pd.read_csv(metadata_path + pid + '.csv')
    p_df = df[df['problem_id'] == pid]
    p_df = p_df.sort_values('original_id')
    problem_df.set_index('submission_id', inplace=True, drop=True)
    problem_df = problem_df.loc[p_df.original_id].sort_index()
    return all(problem_df.status.values == p_df.original_status.values)

for pid in tqdm(problem_list_df.index.unique()):
    if not check(pid):
        print(pid)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4053/4053 [01:09<00:00, 57.94it/s]


So we can see that after running this preprocessing function on all the problems we are left with 100,000 samples of pairs of source code files of the form (error, successful) for all the languages in the dataset. Now the error messages that are provided in this dataset don't look that useful, so in the next steps I will attempt to improve the error messages by running the source code on sample inputs or using compilers, code check tools etc.

### Task 1.2: Analyze source code files
- NOTE: I saw some files that are code golfs, maybe they should be removed from the dataset
- TODO: ?

# Run Code Sources

### Task 1.1: Get error type description
- run the suspicious source code on the sample input
- if there is no sample input: There is a missing value => make the error class NAN
- if it results in error: parse the error name and details
- if there is no error: (there might be an edge case) There is a missing value => make the error class NAN
- return new values (old error type, error class, error desc, return code)

#### Progress

- NOTE: Made a function for python, need one more for C and one C++
- NOTE: How to treat cases where the user debugs messages to console
- NOTE: Some of the programs don't crash on the sample input, but are still suspicious (No error class)
- NOTE: How to treat code golfs?

- TODO: Run the original submission to get a more detailed error on the sample input/output (for C and C++)
- TODO: Filter some of the error types like grouping similar error into the same category
- TODO: Give proper name to number errors

In [37]:
# Load the problem list table as a csv
problem_list_df = pd.read_csv(metadata_path + 'problem_list.csv')
problem_list_df.set_index('id', inplace=True)

problem_list_df['time_limit'].fillna(1.0, inplace=True)
problem_list_df['time_limit_s'] = problem_list_df['time_limit'] / 1000

df = pd.read_csv(generated_pairs_path)

# Eventually I will want to load all languages here
p_df = df[(df['original_language'] == 'Python3') | 
           (df['original_language'] == 'Python') | 
           (df['original_language'] == 'Python (3.8.2)') | 
           (df['original_language'] == 'C')
           # TODO: ADD C AND C++ HERE after testing in sandbox
         ]
p_df

,original_id,changed_id,original_line,changed_line,original_status,original_language,problem_id,language,filename_ext
0,s381898998,s865898267,9,9,Wrong Answer,C,p00000,C,c
1,s749240628,s634652749,11,11,Wrong Answer,C,p00000,C,c
2,s607203127,s502818484,9,9,Wrong Answer,C,p00000,C,c
3,s719001213,s042257533,9,9,Wrong Answer,C,p00000,C,c
4,s066960572,s577188800,10,10,Wrong Answer,C,p00000,C,c
...,...,...,...,...,...,...,...,...,...
102394,s456383760,s016668249,1,1,Wrong Answer,Python (3.8.2),p04029,Python,py
102480,s675922082,s965274869,2,2,Runtime Error,Python (3.8.2),p04043,Python,py
102481,s686089965,s644653229,1,1,Wrong Answer,Python (3.8.2),p04043,Python,py
102505,s075564224,s960630596,1,1,Runtime Error,Python (3.8.2),p04044,Python,py


In [38]:
def handle_process(command, input, timeout):
    process = subprocess.Popen(command,
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE, 
                               encoding="utf-8", 
                               errors="ignore")
    try:
        output, error = process.communicate(input, timeout)
        return output, error, process.returncode
    except subprocess.TimeoutExpired:
        process.kill()
        process.communicate()
        raise TimeoutError

def exec_python(file_path, input=None, timeout=2.0):
    return handle_process(['python3', file_path], input, timeout)

def exec_c(file_path, input=None, timeout=2.0):    
    exec_path = '/tmp/a.out'
    process = subprocess.Popen(['gcc', file_path, '-lm', '-w', '-O3','-o', exec_path],
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE, 
                               encoding="utf-8", 
                               errors="ignore")
    output, error = process.communicate()
    returncode = process.returncode
    if returncode != 0:
        return '', error, returncode
    return handle_process([exec_path], input, timeout)
    
def exec_file(file_path, input=None, timeout=2.0, language=None):
    if language == 'Python':
        return exec_python(file_path, input, timeout)
    if language == 'C':
        return exec_c(file_path, input, timeout)
    else:
        raise NotImplementedError

In [239]:
loop = tqdm(p_df.iterrows(), total=len(p_df))

errs = []

for _id, p_sub in loop:
    original_id = p_sub.loc['original_id']
    original_line = p_sub.loc['original_line']
    problem_id = p_sub.loc['problem_id']
    language = p_sub.loc['language']
    extension = p_sub.loc['filename_ext']
    
    file_path = id2submission(problem_id, language, original_id, extension)
    
    input_path = id2inout(problem_id, name='input')
    output_path = id2inout(problem_id, name='output')
    
    input = None
    if os.path.exists(input_path):
        with open(input_path, 'r') as f:
            input = f.read()
    
    timeout = problem_list_df.loc[problem_id]['time_limit_s'] * 2
    try:
        output, error, returncode = exec_file(file_path, input, timeout, language)
    except TimeoutError:
        returncode = 1
        error = 'TLEError: Time limit exceeded'
        if not input:
            returncode = 0
            error = 'NotAnError: Missing input caused TLE'
    
    errs.append((_id, output, error, returncode))

100%|██████████████████████████████████████████████████████████████████████████| 27959/27959 [59:47<00:00,  7.79it/s]


In [247]:
with open('../temp.pkl', 'wb') as f:
    pickle.dump(errs, f)

In [42]:
with open('../temp.pkl', 'rb') as f:
    errs = pickle.load(f)

In [43]:
def extract_error_class_python(error, returncode):
    rs =  '|'.join([
        r'^(\w*Error):.*',
        r'(\w*Warning):.*',
    ])
    
    p_class = re.compile(rs, re.MULTILINE)
    error_class = p_class.findall(error)
    if not error_class:
        return str(returncode)
    return functools.reduce(lambda acc, x: acc or x, error_class[0], None)

def extract_error_class_extra_python(error, returncode):
    rs =  '|'.join([
        r'^(\w*Error:.*).*',
        r'(\w*Warning:.*).*',
    ])
    
    p_class_extra = re.compile(rs, re.MULTILINE)
    error_class_extra = p_class_extra.findall(error)
    if not error_class_extra:
        return error
    return functools.reduce(lambda acc, x: acc or x, error_class_extra[0], None) 

def extract_error_class_c(error, returncode):
    return str(returncode)

def extract_error_class_extra_c(error, returncode):
    rs = '|'.join([
        r'(undefined reference .*)',
        r'(\*\*\* stack smashing detected \*\*\*: terminated)',
        r'(munmap_chunk\(\): .*)', 
        r'(segmentation fault \(core dumped\))',
        r'(error: .*)',
        r'(relocation truncated to fit: .*)',
        r'(sysmalloc: .*)',
        r'(malloc\(\): .*)',
        r'(free\(\): .*)',
    ])
    p_class_extra = re.compile(rs, re.MULTILINE)
    error_class_extra = p_class_extra.findall(error)
    if not error_class_extra:
        return error
    return functools.reduce(lambda acc, x: acc or x, error_class_extra[0], None)

def extract_error_class(row):
    language, error, returncode = row
    if language == 'C':
        return extract_error_class_c(error, returncode)
    if language == 'Python':
        return extract_error_class_python(error, returncode)
    return None
    
def extract_error_class_extra(row):
    language, error, returncode = row
    if language == 'C':
        return extract_error_class_extra_c(error, returncode)
    if language == 'Python':
        return extract_error_class_extra_python(error, returncode)
    return None

In [44]:
errs_df = pd.DataFrame(errs, columns=['index', 'output', 'error', 'returncode']).set_index('index')
errs_df.index.name = None

p_error_df = pd.concat([p_df, errs_df[[ 'output', 'error', 'returncode']]], axis=1)

p_error_df['error_class'] = p_error_df[['language', 'error', 'returncode']].apply(extract_error_class, axis=1)
p_error_df['error_class_extra'] = p_error_df[['language', 'error', 'returncode']].apply(extract_error_class_extra, axis=1)

p_error_df

,original_id,changed_id,original_line,changed_line,original_status,original_language,problem_id,language,filename_ext,output,error,returncode,error_class,error_class_extra
0,s381898998,s865898267,9,9,Wrong Answer,C,p00000,C,c,1X1=1\n1X2=2\n1X3=3\n1X4=4\n1X5=5\n1X6=6\n1X7=...,,0,0,
1,s749240628,s634652749,11,11,Wrong Answer,C,p00000,C,c,1x1=1\n1x2=2\n1x3=3\n1x4=4\n1x5=5\n1x6=6\n1x7=...,,0,0,
2,s607203127,s502818484,9,9,Wrong Answer,C,p00000,C,c,1x1=1\n1x2=2\n1x3=3\n1x4=4\n1x5=5\n1x6=6\n1x7=...,,0,0,
3,s719001213,s042257533,9,9,Wrong Answer,C,p00000,C,c,1x1\n1x2\n1x3\n1x4\n1x5\n1x6\n1x7\n1x8\n1x9\n2...,,0,0,
4,s066960572,s577188800,10,10,Wrong Answer,C,p00000,C,c,1*1=1\n1*2=2\n1*3=3\n1*4=4\n1*5=5\n1*6=6\n1*7=...,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102394,s456383760,s016668249,1,1,Wrong Answer,Python (3.8.2),p04029,Python,py,3\n,,0,0,
102480,s675922082,s965274869,2,2,Runtime Error,Python (3.8.2),p04043,Python,py,,"Traceback (most recent call last):\n File ""/h...",1,AttributeError,AttributeError: 'list' object has no attribute...
102481,s686089965,s644653229,1,1,Wrong Answer,Python (3.8.2),p04043,Python,py,NO \n,,0,0,
102505,s075564224,s960630596,1,1,Runtime Error,Python (3.8.2),p04044,Python,py,,"Traceback (most recent call last):\n File ""/h...",1,NameError,NameError: name 'n' is not defined


In [45]:
p_error_df['error_class'].value_counts()

0                      20429
1                       2948
SyntaxError             2209
NameError                509
TypeError                457
-11                      450
NotAnError               350
EOFError                 104
AttributeError            91
ValueError                82
IndentationError          58
-6                        52
IndexError                49
ModuleNotFoundError       34
TLEError                  23
-8                        22
SyntaxWarning             19
TabError                  17
ImportError               11
KeyError                   6
UnboundLocalError          6
255                        5
ZeroDivisionError          4
9                          3
4                          3
7                          2
20                         2
2                          2
OverflowError              2
RuntimeError               2
3                          1
6                          1
-7                         1
11                         1
12            